# DATA 304 • Module 6 Exercises: SQL and MongoDB

Do not change anything in cells other than writing your code in designated area.

## Part A — SQL with SQLite
You will load a small relational schema into SQLite and write queries using `SELECT`, `JOIN`, `GROUP BY`, `HAVING`, and `ORDER BY`.

In [1]:
# ASSIGNMENT CELL: SETUP
# create a fresh SQLite database in-memory and seed tables
import pandas as pd
import sqlite3 as sql
conn = sql.connect(':memory:')
cur = conn.cursor()
cur.executescript('''
PRAGMA foreign_keys = ON;
DROP TABLE IF EXISTS departments;
DROP TABLE IF EXISTS employees;
DROP TABLE IF EXISTS sales;
CREATE TABLE departments (id INTEGER PRIMARY KEY, name TEXT NOT NULL);
CREATE TABLE employees (id INTEGER PRIMARY KEY, name TEXT NOT NULL, title TEXT, salary INTEGER, dept_id INTEGER NOT NULL REFERENCES departments(id));
CREATE TABLE sales (id INTEGER PRIMARY KEY, emp_id INTEGER NOT NULL REFERENCES employees(id), item TEXT NOT NULL, amount REAL NOT NULL, sold_at DATE NOT NULL);
''')
cur.executemany('INSERT INTO departments(id, name) VALUES (?,?)', [(1,'Engineering'),(2,'Sales'),(3,'HR')])
cur.executemany('INSERT INTO employees(id,name,title,salary,dept_id) VALUES (?,?,?,?,?)', [
  (1,'Alice','Engineer',105000,1),
  (2,'Bob','Engineer',98000,1),
  (3,'Cara','Sales Rep',90000,2),
  (4,'Dan','Sales Rep',88000,2),
  (5,'Ella','HR Generalist',78000,3)
])
cur.executemany('INSERT INTO sales(emp_id,item,amount,sold_at) VALUES (?,?,?,?)', [
  (3,'Widget-A',1200.0,'2025-08-01'),
  (3,'Widget-B',950.0,'2025-08-02'),
  (4,'Widget-A',500.0,'2025-08-02'),
  (4,'Widget-C',2200.0,'2025-08-03'),
  (3,'Widget-C',1800.0,'2025-08-04')
])
conn.commit()
print('SQLite ready.')

SQLite ready.


### Task A1 — Basic SELECT
List all employee names and titles sorted by name ascending.

In [2]:
# ASSIGNMENT CELL: Q1

# your query starts here
query = """
SELECT name, title 
FROM employees 
ORDER BY name ASC;
"""
# your query ends here

a1 = pd.read_sql_query(query, conn)
a1

,name,title
0,Alice,Engineer
1,Bob,Engineer
2,Cara,Sales Rep
3,Dan,Sales Rep
4,Ella,HR Generalist


### Task A2 — INNER JOIN
Return employee name, department name (use alias `dept`) for all employees.

In [3]:
# ASSIGNMENT CELL: Q2

# your query starts here
query = """
SELECT e.name, d.name AS dept 
FROM employees e JOIN departments d ON e.dept_id=d.id;
"""
# your query ends here

a2 = pd.read_sql_query(query, conn)
a2

,name,dept
0,Alice,Engineering
1,Bob,Engineering
2,Cara,Sales
3,Dan,Sales
4,Ella,HR


### Task A3 — Aggregation
Compute average salary by department. Show columns: `department`, `avg_salary`. Sort by `avg_salary` descending.

In [4]:
# ASSIGNMENT CELL: Q3

# your query starts here
query = """
SELECT d.name AS department, AVG(e.salary) AS avg_salary
FROM employees e JOIN departments d ON e.dept_id=d.id
GROUP BY d.name
ORDER BY avg_salary DESC;
"""
# your query ends here

a3 = pd.read_sql_query(query, conn)
a3

,department,avg_salary
0,Engineering,101500.0
1,Sales,89000.0
2,HR,78000.0


### Task A4 — HAVING
Total sales revenue by sales rep. Only include reps with total revenue ≥ 2000. Columns: `rep`, `total_revenue`.

In [5]:
# ASSIGNMENT CELL: Q4

# your query starts here
query = """
SELECT e.name AS rep, SUM(s.amount) AS total_revenue
FROM employees e JOIN sales s ON e.id=s.emp_id
GROUP BY e.name
HAVING total_revenue>=2000
ORDER BY total_revenue DESC;
"""
# your query ends here

a4 = pd.read_sql_query(query, conn)
a4

,rep,total_revenue
0,Cara,3950.0
1,Dan,2700.0


### Task A5 — Window-like ranking (SQLite workaround)
Get top 2 sales by amount across all reps. Show `rep`, `item`, `amount`.  
Hint: You can get top 2 by ordering and using `LIMIT 2`.

In [6]:
# ASSIGNMENT CELL: Q5

# your query starts here
query = """
SELECT e.name AS rep, s.item, s.amount
FROM sales s JOIN employees e ON s.emp_id=e.id
ORDER BY s.amount DESC
LIMIT 2;
"""
# your query ends here

a5 = pd.read_sql_query(query, conn)
a5

,rep,item,amount
0,Dan,Widget-C,2200.0
1,Cara,Widget-C,1800.0


---
## Part B — MongoDB with PyMongo
You will perform `find()` queries and aggregation pipelines.

In [7]:
# ASSIGNMENT CELL: SETUP
# connect to MongoDB and seed sample data
import mongomock
client = mongomock.MongoClient()
db = client['module6']
users = db['users']
orders = db['orders']
products = db['products']
users.delete_many({}); orders.delete_many({}); products.delete_many({})
products.insert_many([
  {"_id": 101, "sku": "A-100", "name": "Widget-A", "price": 25.0},
  {"_id": 102, "sku": "B-200", "name": "Widget-B", "price": 40.0},
  {"_id": 103, "sku": "C-300", "name": "Widget-C", "price": 55.0}
])
users.insert_many([
  {"_id": 1, "name": "Alice", "city": "NY", "age": 29, "tags": ["python","sql"]},
  {"_id": 2, "name": "Bob", "city": "LA", "age": 41, "tags": ["sales"]},
  {"_id": 3, "name": "Cara", "city": "NY", "age": 35, "tags": ["python","mongodb"]}
])
orders.insert_many([
  {"_id": 5001, "user_id": 1, "items": [{"product_id": 101, "qty": 2}, {"product_id": 102, "qty": 1}], "status": "shipped"},
  {"_id": 5002, "user_id": 2, "items": [{"product_id": 103, "qty": 1}], "status": "processing"},
  {"_id": 5003, "user_id": 3, "items": [{"product_id": 101, "qty": 1}, {"product_id": 103, "qty": 2}], "status": "delivered"}
])
print('MongoDB ready. Collections: users, orders, products')


MongoDB ready. Collections: users, orders, products


### Task B1 — Basic `find()`
Return users aged ≥ 30 who live in `NY`. Project only `name` and `age` (exclude `_id`). Sort by `age` descending.

In [8]:
# ASSIGNMENT CELL: Q6

# your query starts here
cursor = users.find(
    {'age':{'$gte':30}, 'city':'NY'}, 
    {'_id':0,'name':1,'age':1}
).sort('age', -1)
# your query ends here

b1 = pd.DataFrame(list(cursor))
b1

,name,age
0,Cara,35


### Task B2 — `$in`, `$and`
Return users whose `city` is in {`NY`, `LA`} and who have the tag `python`.

In [9]:
# ASSIGNMENT CELL: Q7

# your query starts here
cursor = users.find(
    {
        'city':{'$in':['NY','LA']}, 
        'tags':{'$in':['python']}
    }
)
# your query ends here

b2 = pd.DataFrame(list(cursor))
b2

,_id,name,city,age,tags
0,1,Alice,NY,29,"[python, sql]"
1,3,Cara,NY,35,"[python, mongodb]"


### Task B3 — Aggregation: average age by city
Pipeline: `$match` (age ≥ 30) → `$group` by `city` with `avgAge: {$avg: "$age"}` → `$project` city and `avgAge`.

In [10]:
# ASSIGNMENT CELL: Q8

# your query starts here
cursor = users.aggregate([
  {'$match':{'age':{'$gte':30}}},
  {'$group':{'_id':'$city','avgAge':{'$avg':'$age'}}},
  {'$project':{'_id':0,'city':'$_id','avgAge': '$avgAge'}}
])
# your query ends here

b3 = pd.DataFrame(list(cursor))
b3

,city,avgAge
0,LA,41.0
1,NY,35.0


### Task B4 — Join-like aggregation with `$lookup`
For each SKU, compute total quantity and total revenue across all orders.

Pipeline hint:
1. `$unwind: "$items"`
2. `$lookup` to `products` on `items.product_id = _id`
3. `$unwind: "$prod"`
4. `$group` by `$prod.sku` with fields: `qty: {$sum: "$items.qty"}`, `revenue: {$sum: {$multiply: ["$items.qty", "$prod.price"]}}`
5. `$project` as `{_id: 0, sku: "$_id", qty: 1, revenue: 1}`


In [11]:
# ASSIGNMENT CELL: Q9

# your query starts here
cursor = orders.aggregate([
  {'$unwind':'$items'},
  {'$lookup':{'from':'products','localField':'items.product_id','foreignField':'_id','as':'prod'}},
  {'$unwind':'$prod'},
  {'$group':{'_id':'$prod.sku','qty':{'$sum':'$items.qty'},'revenue':{'$sum':{'$multiply':['$items.qty','$prod.price']}}}},
  {'$project':{'_id':0,'sku':'$_id','qty':1,'revenue':1}},
  {'$sort':{'sku':1}}
])
# your query ends here

b4 = pd.DataFrame(list(cursor))
b4

,qty,revenue,sku
0,3,75.0,A-100
1,1,40.0,B-200
2,3,165.0,C-300
